In [1]:
import numpy as np

In [2]:
def fourier_coeff(f_eval,  T):
    return np.fft.fft(f_eval, axis=0) * T / len(f_eval)

In [ ]:
N = 20   # number of input functions n= 1, 2, ..., N
num_exp = 5 #number of times for each N (for avergaing error)


d = 2**14 # grid points

t_left = -.5
t_right = .5

T = t_right - t_left

t = np.linspace(t_left,t_right,d)
noise = .05


In [ ]:
beta = .7
gamma =  .1
eta = .1
alpha_sup = (2*beta -1 - gamma) / 2*(gamma + eta)
print("alpha <" , alpha_sup)
alpha = .9 * alpha_sup

# lamb =  N ** (-  1  / (2* alpha + 2)) # regularization parameter

In [ ]:
def sin_polin(t,alpha,M):
    return sum((2* np.sin(2*j*np.pi*t) / (j**alpha)) for j in range(1,M+2))

def fourier_expon(t,alpha,M):
    return sum((np.exp(1j*2*k*np.pi*t) / (k**alpha)) for k in range(1,M+2)) + sum((np.exp(1j*2*k*np.pi*t) / (k**alpha)) for k in range(-M,0))



#CHOOSE ground truth function and obs. noise         y_i = x_i * true_w + noise
sum_until = 30  # index M of truncation of the fourier series
true_w = sin_polin(t,beta,sum_until)                        

##fourier coefficients of w_true
true_w_fourier = fourier_coeff(true_w, T)  # normalized by the number of grid points

#for time-loc inputs
delta = 0.076

#for freq-loc inputs
L_max = sum_until


In [ ]:
#To draw samples from the distribution rho_l = l^-beta

def power_law_samples(N, L, eta):
    """
    Generates N samples drawn from a power law probability distribution on the integers
    with exponent eta.
    
    Args:
    - N: an integer specifying the number of samples to generate
    -L: the samples are drawn in the interval [1,L+1]
    - eta: a positive float specifying the exponent of the power law
    
    Returns:
    - A numpy array of N integers drawn from the power law distribution
    """
    # Define the power law probability density function
    def p(x):
        return 1.0 / (x**eta)
    
    # Define the normalization constant
    Z = sum([p(l) for l in range(1, L+1)])
    
    # Generate the samples
    samples = []
    while len(samples) < N:
        # Choose a random integer in the range [1, L+1] with probability proportional to p(l)
        n = np.random.choice(range(1, L+1), p=[p(l)/Z for l in range(1, L+1)])
        samples.append(n)
    
    return np.array(samples)

In [ ]:
#kernel coefficients

kernel_coeff = np.zeros(t.size, dtype=complex)
kernel_coeff[0] = 1
for k in range(1,t.size):
    kernel_coeff[k] = k**(-gamma) 